# 제4 고지 : 신경망 만들기 
## STEP 47 : 소프트맥스 함수와 교차 엔트로피 오차

이전 단계에서는 회귀문제를 풀어봤는데, 앞으로는 `multi-class classification` 에 대해 알아본다.


### 47.1 슬라이스 조작 함수

구체적인 구현은 추후 부록.B를 참고하도록 하고, 사용법에 대해서만 우선 살펴본다.

<p align='center'>
    <img src='../assets/그림 47-1.png' align='center' width='50%'>
</p>

`get_item()` 은 `Variable` 의 다차원 배열 중에서 일부를 슬라이스 하여 뽑아준다. 즉 **슬라이스는 데이터의 일부를 수정하지 않고 그대로 전달하는 역할**이므로, **데이터가 추출된 위치에만 기울기를 1로 설정하고 이외에는 0으로 설정**한다.


In [2]:
import sys
sys.path.append("..")

import numpy as np 
from dezero import Variable
import dezero.functions as F 

x = Variable(np.array([[1,2,3],[4,5,6]]))
y = F.get_item(x,1)
print("슬라이스 : ", y)
print("="*10)
y.backward()
print("기울기 : ",x.grad)
print("="*10)

# 또한 같은 인덱스를 반복 지정하여 동일한 원소를 여러 번 빼낼 수 있다.
indices = np.array([0,0,1])
y = F.get_item(x,indices)
print(y)

print("="*10)
# 또한 dezero/core.py 에서 Variable.__getitem__ = dezero.functions.get_item 로 다음과 같이 슬라이스 연산이 가능하며 역전파 역시 잘 작동한다.
y = x[1]
print(y)

y = x[:,2]
print(y)



슬라이스 :  variable([4 5 6])
기울기 :  variable([[0 0 0]
          [1 1 1]])
variable([[1 2 3]
          [1 2 3]
          [4 5 6]])
variable([4 5 6])
variable([3 6])


### 47.2 소프트맥스 함수
신경망의 출력은 단순한 수치인데, 다중 클래스 분류를 위해 이 수치를 '확률'로 변환할 필요가 있다.  
다음 softmax function을 이용하면 확률로 변환 가능하다. 

$$
p_k = \frac{e^{y_k}}{\sum_{i=1}^n e^{y_i}}
$$

구체적으로 살펴보면, 합수의 입력 $y_k$  총 $n$ 개 (클래스 수) 라고 가정하면 $p_1+p_2+\cdots+p_n = 1 $ 이 성립하여 $(p_1,p_2,\cdots,p_n)$의 원소 각각을 확률로 해석할 수 있다.


<p align='center'>
    <img src='../assets/그림 47-2.png' align='center' width='70%'>
</p>

이해를 돕기 위해 위의 그림을 보면 소프트맥스 함수를 적용하고나면 확률 $p_1,p_2,p_3$로 표현되고 합이 1 임을 알 수 있다.

<span style='background-color : #ffdce0'>💡<b>주의해야할 것은 소프트맥스 함수는 지수함수로 이루어져 있기 때문에 너무 커지거나 작아질 수 있다. 따라서 오버플로우 문제를 잘 다뤄줘야 하는데 실제 구현시에는 $p_k = \frac{Ce^{y_k}}{C\sum_{i=1}^n e^{y_i}}=\frac{e^{y_k}+\log C}{\sum_{i=1}^n e^{y_i}+\log C}=\frac{e^{y_k}+C'}{\sum_{i=1}^n e^{y_i}+C'}$와 같이   지수함수계산시에는 어떤 정수를 더하거나 빼더라도 결과는 바뀌지 않는 다는 사실을 이용해 최대값을 빼서 오버플로우를 막아준다.</b></span>


In [3]:
from dezero import Variable, as_variable
from dezero.models import MLP
import dezero.functions as F 

def softmax1d(x):
    x = as_variable(x)
    y = F.exp(x)
    sum_y = F.sum(y)
    return y / sum_y

x = Variable(np.array([[0.2,-0.4]]))
model = MLP((10,3))
y = model(x)
p = softmax1d(y)
print("신경망 출력 : ",y)
print("소프트 맥스 적용 후 : ",p)


신경망 출력 :  variable([[ 0.12806286 -0.08324128  0.3620511 ]])
소프트 맥스 적용 후 :  variable([[0.32539823 0.26341892 0.41118285]])


이어서 **배치 데이터에서도 적용 가능**하게 하려면 다음과 같이 함수를 수정한다.
```python
def softmax_simple(x,axis=1):
    x = as_variable(x)
    y = exp(x) # DeZero exp 함수
    sum_y = sum(y,axis=axis,keepdims=True)
    return y / sum_y 
```

추가적으로, 더 나은 구현 방식을 위해 `Function`을 상속하여 `Softmax` 클래스를 구현해보자.
```python
class Softmax(Function):
    def __init__(self, axis=1):
        self.axis = axis

    def forward(self, x):
        y = x - x.max(axis=self.axis, keepdims=True) # 오버플로우 방지를 위해 최대값을 빼준다.
        y = np.exp(y)
        y /= y.sum(axis=self.axis, keepdims=True)
        return y

    def backward(self, gy):
        y = self.outputs[0]()
        gx = y * gy
        sumdx = gx.sum(axis=self.axis, keepdims=True)
        gx -= y * sumdx
        return gx

def softmax(x, axis=1):
    return Softmax(axis)(x)
```

### 47.3 교차 엔트로피 오차 
선형회귀에서는 `MSE` 를 손실함수로 사용했다면, multi-class classification  에서는 `교차 엔트로피 오차 (cross entropy error)` 를 손실함수로 일반적으로 사용한다. 

$$
L = - \sum_k t_k\log p_k
$$

여기서 $t_k$ 는 정답 데이터의 $k$차원의 값으로 정답이면 1, 아니면 0으로 표현되는 원핫 벡터로 실제분포이고,  $p_k$ 는 신경망에서 소프트맥스를 적용한 함수 값으로 예측 분포이다.
 

이를 조금더 간단하게 표현하면 다음과 같다. 
$$
L= -\log\mathbf{p}[t]
$$

구체적으로 살펴보면, 만약 $\mathbf{t}=(0,0,1),\mathbf{p}=(p_0,p_1,p_2)$라고 하면 $L=-\log p_2$이므로 정답클래스에 해당하는 번호의 확률 $\mathbf{p}$를 추출하는것과 같으므로, 위와 같이 표현이 가능하다.

<span style='background-color : #ffdce0'>💡<b>이번 설명은 데이터가 하나인 경우에 해당하고 만약 데이터가 $N$개라면 평균 교차엔트로피 오차를 구해야 한다.</b></span>

이제 `소프트맥스 함수` 와 `교차 엔트로피 오차` 를 한꺼번에 수행하는 `softmax_cross_entropy_simple(x,t)` 함수를 구현해보자.


```python
def softmax_cross_entropy_simple(x, t):
    '''
    x : 신경망에서 소프트맥수 적용전의 출력값
    t : 정답 데이터로 정답 클래스의 번호 (원핫 벡터가 아니다)
    '''
    x, t = as_variable(x), as_variable(t)
    N = x.shape[0]
    p = softmax(x)
    p = clip(p, 1e-15, 1.0)  # To avoid log(0), 만약 p가 0이면 1e-15로 대체, 1.0을 넘으면 1.0
    log_p = log(p)
    tlog_p = log_p[np.arange(N), t.data]
    y = -1 * sum(tlog_p) / N
    return y
```



In [4]:
from dezero import Variable, as_variable
from dezero.models import MLP
import dezero.functions as F 


x = np.array([[0.2,-0.4],[0.3,0.5],[1.3,-3.2],[2.1,0.3]])
t = np.array([2,0,1,0])
model = MLP((10,3))
y = model(x)
loss = F.softmax_cross_entropy(y,t)
print(loss)

variable(0.9820149162942775)
